## Importamos las librerias necesarias

In [1]:
import json
import pandas as pd
from pandas import json_normalize
import ast
import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
import nltk
import re
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from textblob import TextBlob
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.metrics.pairwise import cosine_similarity


nltk.download('vader_lexicon')
# Download necessary NLTK resources if not available
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Error loading vader_lexicon: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1002)>
[nltk_data] Error loading vader_lexicon: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1002)>
[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1002)>
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1002)>


False

## Importamos el primer JSON y realizamos un breve análisis exploratorio de datos.

Seguimos los mismos pasos para cada dataset:

1. Importamos el JSON: Cargamos los datos desde el primer archivo JSON.

2. Análisis exploratorio: Realizamos un análisis inicial de los datos para comprender su estructura y contenido.

3. Eliminación de columnas no útiles: Identificamos las columnas que no serán útiles para nuestro análisis y las eliminamos del conjunto de datos.

4. Adecuación del formato: Aseguramos que el formato de las columnas restantes sea apropiado para nuestro análisis. Esto puede incluir la conversión de tipos de datos o la normalización de formatos.

5. Búsqueda de duplicados y nulos: Identificamos y gestionamos los registros duplicados y los valores nulos en el conjunto de datos.

Este proceso nos permitirá preparar los datos para análisis posteriores de manera más efectiva.


# Games.

In [21]:
# Ruta del archivo JSON
file_path = 'steam_games.json'

# Lista para almacenar objetos JSON parseados
json_data_list = []

# Abrir el archivo JSON, leer su contenido y cerrarlo
with open(file_path, 'r') as f:
    for line in f:
        # Analizar cada línea como un objeto JSON
        json_data = json.loads(line)
        json_data_list.append(json_data)

# Ahora, 'json_data_list' contiene una lista de objetos JSON del archivo


In [22]:
# Convierte la lista de diccionarios (JSON objects) en un DataFrame
df_games = pd.DataFrame(json_data_list)


In [23]:
df_games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"


In [24]:
df_games.tail()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"
120444,NaN,NaN,Maze Run VR,NaN,http://store.steampowered.com/app/681550/Maze_...,NaN,"[Early Access, Adventure, Indie, Action, Simul...",http://steamcommunity.com/app/681550/reviews/?...,"[Single-player, Stats, Steam Leaderboards, HTC...",4.99,True,681550,NaN


In [25]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     24083 non-null  object
 1   genres        28852 non-null  object
 2   app_name      32133 non-null  object
 3   title         30085 non-null  object
 4   url           32135 non-null  object
 5   release_date  30068 non-null  object
 6   tags          31972 non-null  object
 7   reviews_url   32133 non-null  object
 8   specs         31465 non-null  object
 9   price         30758 non-null  object
 10  early_access  32135 non-null  object
 11  id            32133 non-null  object
 12  developer     28836 non-null  object
dtypes: object(13)
memory usage: 11.9+ MB


In [26]:
df_games.columns

Index(['publisher', 'genres', 'app_name', 'title', 'url', 'release_date',
       'tags', 'reviews_url', 'specs', 'price', 'early_access', 'id',
       'developer'],
      dtype='object')

### Limpieza, formateo y normalización. 

In [27]:
# Lista de columnas a eliminar
to_drop = ['publisher', 'app_name', 'reviews_url', 'early_access', 'url', 'specs', 'price', 'developer']

# Eliminar las columnas especificadas del DataFrame
df_games = df_games.drop(to_drop, axis=1)


In [28]:
# Lista de columnas a procesar
cols = ['genres', 'tags']

# Itera a través de las columnas
for i in cols:
    # Convierte los valores de la columna en cadenas de texto
    df_games[i] = df_games[i].astype(str)
    
    # Elimina los corchetes '[' y ']' de los valores de la columna
    df_games[i] = df_games[i].str.replace('[', '').str.replace(']', '')



In [36]:
# Convierte la columna 'id' a tipo int
df_games['id'] = df_games['id'].astype(int)

# Convierte la columna 'release_date' a tipo datetime
df_games['release_date'] = pd.to_datetime(df_games['release_date'], errors='coerce')


In [30]:
df_games['title'] = df_games['title'].str.lower()

In [559]:
df_games

,genres,title,release_date,tags,id
88310,"'Action', 'Casual', 'Indie', 'Simulation', 'St...",lost summoner kitty,2018-01-04,"'Strategy', 'Action', 'Indie', 'Casual', 'Simu...",761140
88311,"'Free to Play', 'Indie', 'RPG', 'Strategy'",ironbound,2018-01-04,"'Free to Play', 'Strategy', 'Indie', 'RPG', 'C...",643980
88312,"'Casual', 'Free to Play', 'Indie', 'Simulation...",real pool 3d - poolians,2017-07-24,"'Free to Play', 'Simulation', 'Sports', 'Casua...",670290
88313,"'Action', 'Adventure', 'Casual'",弹炸人2222,2017-12-07,"'Action', 'Adventure', 'Casual'",767400
88315,"'Action', 'Adventure', 'Simulation'",battle royale trainer,2018-01-04,"'Action', 'Adventure', 'Simulation', 'FPS', 'S...",772540
...,...,...,...,...,...
120439,"'Action', 'Adventure', 'Casual', 'Indie'",kebab it up!,2018-01-04,"'Action', 'Indie', 'Casual', 'Violent', 'Adven...",745400
120440,"'Casual', 'Indie', 'Simulation', 'Strategy'",colony on mars,2018-01-04,"'Strategy', 'Indie', 'Casual', 'Simulation'",773640
120441,"'Casual', 'Indie', 'Strategy'",logistical: south africa,2018-01-04,"'Strategy', 'Indie', 'Casual'",733530
120442,"'Indie', 'Racing', 'Simulation'",russian roads,2018-01-04,"'Indie', 'Simulation', 'Racing'",610660


### Duplicados y Nulos.

In [31]:
df_games.duplicated()

0         False
1          True
2          True
3          True
4          True
          ...  
120440    False
120441    False
120442    False
120443    False
120444    False
Length: 120445, dtype: bool

In [32]:
df_games = df_games.drop_duplicates()

In [33]:
df_games.isnull()

,genres,title,release_date,tags,id
0,False,True,True,False,True
88310,False,False,False,False,False
88311,False,False,False,False,False
88312,False,False,False,False,False
88313,False,False,False,False,False
...,...,...,...,...,...
120440,False,False,False,False,False
120441,False,False,False,False,False
120442,False,False,False,False,False
120443,False,False,False,False,False


In [34]:
df_games.isnull().sum()

genres             0
title           2050
release_date    2067
tags               0
id                 2
dtype: int64

In [35]:
df_games = df_games.dropna()

### Exportar el DataFrame a CSV


In [82]:
# Exporta el DataFrame df_games a un archivo CSV en el directorio actual
df_games.to_csv('df_games.csv', index=False)


# Reviews.

In [37]:
# Ruta al archivo JSON descomprimido
reviews = 'user_reviews.json'

# Lista para almacenar objetos JSON parseados
data_list = []

# Abrir el archivo JSON, leer su contenido y cerrarlo
with open(reviews, 'r', encoding='utf8') as f:
    for line in f:
        try:
            # Utilizamos Ast.literal_eval para convertir la línea en un diccionario
            json_data = ast.literal_eval(line)
            data_list.append(json_data)
        except ValueError as e:
            # Si ocurre un error al parsear la línea, imprimir un mensaje de error y continuar
            print(f"Error: {line}")
            continue

# Ahora, 'data_list' contiene una lista de objetos JSON del archivo


In [38]:
# Crear un DataFrame a partir de la lista data_list
df_reviews = pd.DataFrame(data_list)


In [39]:
df_reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


### Limpieza, formateo y normalización. 

In [40]:
# Normalize the 'reviews' column while preserving 'user_id' and 'user_url' columns
df_reviews_normalized = pd.json_normalize(df_reviews.to_dict('records'), 'reviews', ['user_id', 'user_url'])


In [41]:
df_reviews_normalized

,funny,posted,last_edited,item_id,helpful,recommend,review,user_id,user_url
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,js41637,http://steamcommunity.com/id/js41637
4,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,js41637,http://steamcommunity.com/id/js41637
...,...,...,...,...,...,...,...,...,...
59300,,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...,76561198312638244,http://steamcommunity.com/profiles/76561198312...
59301,,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...,76561198312638244,http://steamcommunity.com/profiles/76561198312...
59302,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...,LydiaMorley,http://steamcommunity.com/id/LydiaMorley
59303,,Posted July 20.,,730,No ratings yet,True,:D,LydiaMorley,http://steamcommunity.com/id/LydiaMorley


In [42]:
columns_drop = ['last_edited', 'user_url', 'helpful', 'funny']
df_reviews_normalized = df_reviews_normalized.drop(columns_drop, axis=1)

In [43]:
df_reviews_normalized["review"].tail(10)

59295    I cried in the end its so sadding ]'; I wish l...
59296    Gra naprawdę fajna.Ale jest kilka rzeczy do kt...
59297                                            Well Done
59298    this is a very fun and nice 80s themed shooter...
59299    if you liked Half life i would really recommen...
59300    a must have classic from steam definitely wort...
59301    this game is a perfect remake of the original ...
59302    had so much fun plaing this and collecting res...
59303                                                   :D
59304                                       so much fun :D
Name: review, dtype: object

### Duplicados y Nulos.

In [44]:
df_reviews_normalized.duplicated().sum()

874

In [45]:
df_reviews_normalized = df_reviews_normalized.drop_duplicates()

In [46]:
df_reviews_normalized.isnull().sum()

posted       0
item_id      0
recommend    0
review       0
user_id      0
dtype: int64

### Exportar el DataFrame a CSV


In [ ]:
df_reviews_normalized.to_csv('df_reviews.csv', index=False)

# Items.

In [3]:
# Ruta al archivo JSON descomprimido
file_items = 'users_items.json'

# Lista para almacenar objetos JSON parseados
json_items_list = []

# Abrir el archivo JSON, leer su contenido y cerrarlo
with open(file_items, 'r') as f:
    for line in f:
        # Parsear cada línea como un objeto JSON
        json_items = ast.literal_eval(line)
        json_items_list.append(json_items)

# Ahora, 'json_items_list' contiene una lista de objetos JSON del archivo


In [4]:
df_items = pd.DataFrame(json_items_list)

In [5]:
df_items

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."


In [6]:
df_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


### Limpieza, formateo y normalización. 

In [7]:
# Utilizamos json_normalize para extraer la información anidada en la columna 'items'

df_items_normalized = pd.json_normalize(
    df_items.to_dict('records'),  # Convertir el DataFrame en una lista de registros tipo diccionario
    'items',                       # Nombre de la columna con datos anidados
    ['user_id', 'items_count', 'steam_id', 'user_url']  # Columnas adicionales que se mantendrán
)

In [9]:
columns_to_drop = ["playtime_2weeks", "items_count", "steam_id"]
df_items_normalized = df_items_normalized.drop(columns=columns_to_drop)


In [11]:
df_items_normalized = df_items_normalized.drop(columns= 'user_url')

In [12]:
df_items_normalized

,item_id,item_name,playtime_forever,user_id
0,10,Counter-Strike,6,76561197970982479
1,20,Team Fortress Classic,0,76561197970982479
2,30,Day of Defeat,7,76561197970982479
3,40,Deathmatch Classic,0,76561197970982479
4,50,Half-Life: Opposing Force,0,76561197970982479
...,...,...,...,...
5153204,346330,BrainBread 2,0,76561198329548331
5153205,373330,All Is Dust,0,76561198329548331
5153206,388490,One Way To Die: Steam Edition,3,76561198329548331
5153207,521570,You Have 10 Seconds 2,4,76561198329548331


In [13]:

# Lista de columnas que deseas convertir a tipo de datos int
columnas_a_convertir = ['user_id', 'item_id', 'playtime_forever']

# Iterar a través de las columnas y convertirlas a int, manejando los errores
for columna in columnas_a_convertir:
    df_items_normalized[columna] = pd.to_numeric(df_items_normalized[columna], errors='coerce')

# Ahora, los valores no válidos se establecerán como NaN en esas columnas



### Duplicados y nulos.

In [15]:
df_items_normalized.duplicated().sum()

2019356

In [16]:
df_items_normalized = df_items_normalized.drop_duplicates()

In [17]:
df_items_normalized.isnull().sum()

item_id                  0
item_name                0
playtime_forever         0
user_id             904777
dtype: int64

In [18]:
df_items_normalized = df_items_normalized.dropna()

### Exportamos items en formato parquet.

In [190]:
df_items = pa.Table.from_pandas(df_items_normalized)
pq.write_table(df_items, 'items.parquet') 


# PRIMERA FUNCION

def PlayTimeGenre( genero : str ): Debe devolver año con mas horas jugadas para dicho género.
Ejemplo de retorno: {"Año con más horas jugadas para Género X" : 2013}



In [47]:
def PlayTimeGenre(genero:str, df_games, df_items_normalized):
    # Filtrar el DataFrame df_games para obtener los juegos del género especificado
    games_filtered = df_games[df_games['genres'].str.contains(genero, case=False, na=False)]

    # Filtrar df_items_normalized para obtener filas con item_id similares
    merged_df = pd.merge(df_items_normalized, games_filtered, left_on='item_id', right_on='id', how='inner')

    # Convertir la columna 'release_date' a formato de fecha y obtener el año de lanzamiento
    merged_df['release_date'] = pd.to_datetime(merged_df['release_date'])
    merged_df['release_year'] = merged_df['release_date'].dt.year

    # Agrupar por año de lanzamiento y sumar las horas de juego para cada año
    grouped = merged_df.groupby('release_year')['playtime_forever'].sum()

    # Encontrar el año de lanzamiento con más horas jugadas
    max_year = grouped.idxmax()

    # Devolver el resultado como un diccionario
    return {'Año de lanzamiento con más horas jugadas por género': max_year}



In [48]:
# Llama a la función con el género deseado y los DataFrames correspondientes
genero_deseado = "'Action'"  # Reemplaza "Género X" con el género específico que deseas buscar
resultado = PlayTimeGenre(genero_deseado, df_games, df_items_normalized)
print(resultado)

{'Año de lanzamiento con más horas jugadas por género': 2012.0}


# SEGUNDA FUNCION

def UserForGenre( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.
Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

In [49]:

def UserForGenre(genero: str):
    global df_games
    global df_items_normalized
    # Paso 1: Filtrar el DataFrame df_games por el género especificado
    games_filtrados = df_games[df_games['genres'].str.contains(genero, case=False, na=False)]

    # Paso 2: Filtrar df_items_normalized para obtener las filas con item_id coincidente
    merged_df = pd.merge(df_items_normalized, games_filtrados, left_on='item_id', right_on='id', how='inner')
    merged_df['release_date'] = pd.to_datetime(merged_df['release_date'])
    merged_df['release_year'] = merged_df['release_date'].dt.year

    # Paso 3: Agrupar por user_id y release_year, calcular la suma de horas jugadas
    grouped = merged_df.groupby(['user_id', 'release_year'])['playtime_forever'].sum().reset_index() ###VERRRRRRRRRRRRRRRRRRRRR

    # Paso 4: Encontrar el usuario con más horas jugadas para el género
    max_user = grouped.groupby('user_id')['playtime_forever'].sum().idxmax()

    # Paso 5: Crear una lista de acumulación de horas jugadas por año
    horas_por_anio = grouped[grouped['user_id'] == max_user][['release_year', 'playtime_forever']].to_dict('records')

    return {
        "Usuario con más horas jugadas para Género X": max_user,
        "Horas jugadas": horas_por_anio
    }



In [50]:
# Llama a la función con el género deseado y los DataFrames correspondientes
genero_deseado = "action"  # Reemplaza "Género X" con el género específico que deseas buscar
resultado = UserForGenre(genero_deseado)
print(resultado)

{'Usuario con más horas jugadas para Género X': 7.656119798142522e+16, 'Horas jugadas': [{'release_year': 2003.0, 'playtime_forever': 2069}, {'release_year': 2004.0, 'playtime_forever': 95065}, {'release_year': 2005.0, 'playtime_forever': 3314}, {'release_year': 2006.0, 'playtime_forever': 5548}, {'release_year': 2008.0, 'playtime_forever': 367}, {'release_year': 2009.0, 'playtime_forever': 1428}, {'release_year': 2010.0, 'playtime_forever': 252961}, {'release_year': 2011.0, 'playtime_forever': 3048}, {'release_year': 2012.0, 'playtime_forever': 129287}, {'release_year': 2013.0, 'playtime_forever': 47994}, {'release_year': 2014.0, 'playtime_forever': 247}, {'release_year': 2015.0, 'playtime_forever': 30371}, {'release_year': 2016.0, 'playtime_forever': 2468}]}


# TERCERA FUNCION

### Preparación de Datos y Función UsersRecommend
En esta sección, se realiza la preparación de datos y se crea la función UsersRecommend(year) que devuelve el top 3 de juegos más recomendados por usuarios para un año dado. Estos son los pasos principales:

-Análisis de Sentimiento: Se utiliza NLTK para realizar un análisis de sentimiento en las reseñas de juegos y se asigna un valor de sentimiento (positivo, neutral o negativo) a cada reseña.

-Extracción de Fechas: Se extraen las fechas de las reseñas y se crea una nueva columna llamada 'date'.

-Limpieza y Fusión de Datos: Se realizan varias transformaciones y limpieza de datos en los DataFrames y se fusionan para obtener información completa.

-Función UsersRecommend: Se define la función UsersRecommend(year) que filtra y analiza las reseñas para el año proporcionado y devuelve el top 3 de juegos más recomendados por usuarios con comentarios positivos o neutrales.

Este proceso permite obtener recomendaciones de juegos en función de las reseñas de los usuarios para un año específico.

In [51]:
# Crea una instancia de NLTK's sentiment analyzer
sia = SentimentIntensityAnalyzer()
# Definir una función para preprocesar el texto
def preprocess_text(text):
    # Convertir el texto a minúsculas
    text = text.lower()
    # Eliminar caracteres no alfabéticos y números
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    # Tokenizar el texto
    tokens = nltk.word_tokenize(text)
    # Eliminar palabras vacías
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    # Lematizar las palabras (convertirlas a su forma base)
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    # Unir las palabras procesadas en una cadena
    preprocessed_text = ' '.join(tokens)
    return preprocessed_text

In [52]:
# Aplicar la función de preprocesamiento al DataFrame
df_reviews_normalized['preprocessed_review'] = df_reviews_normalized['review'].apply(preprocess_text)


In [53]:
# Definir una función para asignar el análisis de sentimiento con TextBlob
def assign_sentiment_score(text):
    analysis = TextBlob(text)
    if analysis.sentiment.polarity > 0.2:
        return 2  # Positivo
    elif analysis.sentiment.polarity < -0.2:
        return 0  # Malo
    else:
        return 1  # Neutral

In [54]:
# Aplicar la función de análisis de sentimiento al DataFrame
df_reviews_normalized['sentiment_analysis'] = df_reviews_normalized['preprocessed_review'].apply(assign_sentiment_score)


In [55]:
# Aplicar la función de análisis de sentimiento y crear la nueva columna 'sentiment_analysis'
df_reviews_normalized['sentiment_analysis'] = df_reviews_normalized['review'].apply(assign_sentiment_score)

# Reemplazar la columna original con la nueva columna de análisis de sentimiento
df_reviews_normalized.drop(columns=['review','preprocessed_review'], inplace=True)

# Guardar el DataFrame actualizado en un nuevo archivo CSV
df_reviews_normalized.to_csv('reviews_sentiment.csv', index=False)  # Reemplaza 'tu_archivo_actualizado.csv' con el nombre deseado

In [56]:
import re

# Función para extraer la fecha de la columna 'posted'
def extract_date(posted_text):
    # Utilizar una expresión regular para buscar la fecha en el formato deseado
    date_match = re.search(r'[A-Za-z]+\s\d+,\s\d+', posted_text)
    
    # Si se encontró una fecha, devolverla; de lo contrario, devolver None
    if date_match:
        return date_match.group()
    else:
        return None

# Aplicar la función a la columna 'posted' y crear una nueva columna 'date'
df_reviews_normalized['date'] = df_reviews_normalized['posted'].apply(extract_date)

# Mostrar el DataFrame resultante con la nueva columna 'date'
print(df_reviews_normalized[['posted', 'date']])


                          posted               date
0       Posted November 5, 2011.   November 5, 2011
1          Posted July 15, 2011.      July 15, 2011
2         Posted April 21, 2011.     April 21, 2011
3          Posted June 24, 2014.      June 24, 2014
4      Posted September 8, 2013.  September 8, 2013
...                          ...                ...
59300            Posted July 10.               None
59301             Posted July 8.               None
59302             Posted July 3.               None
59303            Posted July 20.               None
59304             Posted July 2.               None

[58431 rows x 2 columns]


In [57]:
df_reviews_normalized = df_reviews_normalized.drop('posted', axis=1)

In [58]:
df_reviews_normalized['sentiment_analysis'].value_counts()

sentiment_analysis
1    35878
2    17419
0     5134
Name: count, dtype: int64

In [59]:
df_reviews_normalized['item_id'] = df_reviews_normalized['item_id'].astype(int)

In [60]:
df_reviews_normalized.isnull().sum()

item_id                  0
recommend                0
user_id                  0
sentiment_analysis       0
date                  9933
dtype: int64

In [61]:
df_reviews_normalized = df_reviews_normalized.dropna()

### Exportamos el df de reviews con las modificaciones hechas (sentiment analysis, etc) y columnas necesarias a utilizar.

In [131]:
df_reviews_normalized.to_csv('df_reviews.csv')

### Merge para utilizar en la función.

In [62]:
merged_df = df_reviews_normalized.merge(df_games, left_on='item_id', right_on='id', how='inner')


In [64]:
merged_df = merged_df[['date', 'title', 'sentiment_analysis', 'recommend', 'release_date', 'id']]


In [65]:
merged_df

,date,title,sentiment_analysis,recommend,release_date,id
0,"November 5, 2011",killing floor,1,True,2009-05-14,1250
1,"March 30, 2015",killing floor,2,True,2009-05-14,1250
2,"July 12, 2013",killing floor,1,True,2009-05-14,1250
3,"August 13, 2015",killing floor,1,True,2009-05-14,1250
4,"April 5, 2014",killing floor,1,True,2009-05-14,1250
...,...,...,...,...,...,...
41209,"June 11, 2015",aero's quest,1,True,2015-06-08,367780
41210,"June 13, 2015",aero's quest,1,True,2015-06-08,367780
41211,"June 13, 2015",another perspective,0,False,2014-08-08,305920
41212,"August 11, 2015",the howler,2,True,2016-04-21,306040


In [66]:
merged_df.duplicated().sum()

6987

In [69]:
merged_df = merged_df.drop_duplicates()

In [72]:
merged_df.isnull().sum()

date                    0
title                   0
sentiment_analysis      0
recommend               0
release_date          152
id                      0
dtype: int64

### Exportamos merge. 

In [200]:
merged_df.to_csv('merged_data.csv', index=False)

## Tercera Función.

In [73]:
def UsersRecommend(year):
    # Filtra las reseñas por el año proporcionado
    df_filtered = merged_df[merged_df['date'].str.contains(str(year), na=False)]

    # Filtra las reseñas recomendadas y con sentimiento positivo o neutral
    df_filtered = df_filtered[(df_filtered['recommend'] == True) & (df_filtered['sentiment_analysis'] >= 1)]

    # Cuenta cuántas veces se recomienda cada juego
    top_games = df_filtered['title'].value_counts().head(3)

    # Formatea los resultados en el formato deseado
    result = [{"Puesto {}".format(i + 1): game} for i, game in enumerate(top_games.index)]
    return result


In [74]:
# Llama a la función con el año deseado
year = 2010 # Reemplaza con el año deseado
recommendations = UsersRecommend(year)
print(recommendations)


[{'Puesto 1': 'team fortress 2'}, {'Puesto 2': 'killing floor'}, {'Puesto 3': 'alien swarm'}]


# CUARTA FUNCION

def UsersNotRecommend( año : int ): Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)
Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]



In [75]:
def UsersNotRecommend(year):
    # Filtra las reseñas por el año proporcionado
    df_filtered = merged_df[merged_df['date'].str.contains(str(year), na=False)]

    # Filtra las reseñas no recomendadas y con sentimiento negativo
    df_filtered = df_filtered[(df_filtered['recommend'] == False) & (df_filtered['sentiment_analysis'] == 0)]

    # Cuenta cuántas veces no se recomienda cada juego
    top_not_recommended_games = df_filtered['title'].value_counts().head(3)

    # Formatea los resultados en el formato deseado
    result = [{"Puesto {}".format(i + 1): game} for i, game in enumerate(top_not_recommended_games.index)]
    return result


In [76]:
# Llama a la función con el año deseado
year = 2013  # Reemplaza con el año deseado
rec = UsersNotRecommend(year)
print(rec)

[{'Puesto 1': 'call of duty®: ghosts'}, {'Puesto 2': 'team fortress 2'}, {'Puesto 3': 'nether: resurrected'}]


# QUINTA FUNCION

In [77]:
def sentiment_analysis(year):
    # Convertir la columna 'release_date' en un formato de fecha
    merged_df['release_date'] = pd.to_datetime(merged_df['release_date'])

    # Filtrar las reseñas por el año de lanzamiento
    review_year = merged_df[merged_df['release_date'].dt.year == year]
    
    # Contar la cantidad de reseñas para cada categoría de análisis de sentimiento
    count_negative = (review_year['sentiment_analysis'] == 0).sum()
    count_neutral = (review_year['sentiment_analysis'] == 1).sum()
    count_positive = (review_year['sentiment_analysis'] == 2).sum()
    
    # Crear el diccionario de resultados
    result = {
        'Negative': count_negative,
        'Neutral': count_neutral,
        'Positive': count_positive
    }
    
    return result

In [78]:
sentiment_analysis(2015)

/var/folders/5w/6sy3cr4d4t35b_h01s9cd7mr0000gn/T/ipykernel_17188/1563106809.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['release_date'] = pd.to_datetime(merged_df['release_date'])


{'Negative': 384, 'Neutral': 2454, 'Positive': 1058}

## Recomendación de Juegos Basada en Género

Este código implementa un sistema de recomendación de juegos basado en género utilizando la similitud del coseno. A continuación, se describen los pasos clave:

### Paso 1: Preprocesamiento de Datos

Se seleccionan las columnas relevantes del DataFrame `df_games`, incluyendo 'id', 'genres' y 'title'. Además, se realiza una limpieza de los valores en la columna 'genres', eliminando comas y espacios adicionales.

### Paso 2: Codificación One-Hot de Géneros

Se realiza la codificación one-hot de los géneros de los juegos. Esto implica convertir la columna 'genres' en un conjunto de columnas binarias que representan cada género de juego.

### Paso 3: Función de Recomendación

Se define la función `recomendacion_juego(id_de_producto)` que toma un ID de juego como entrada. La función encuentra el vector de género del juego de entrada, calcula la similitud del coseno entre el juego de entrada y todos los juegos, ordena los juegos por similitud y selecciona los 5 más similares. Luego, se excluye el juego de entrada por título y se devuelve una lista de los títulos de los 5 juegos más similares como recomendación.

Este sistema de recomendación permite encontrar juegos similares en función de sus géneros y puede ser útil para los amantes de ciertos géneros de juegos.


In [85]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Paso 1: Preprocesamiento de datos
games = df_games[['id', 'genres', 'title']]

# Limpia los valores de la columna 'genres' eliminando comas y espacios adicionales
games['genres'] = games['genres'].apply(lambda x: ', '.join([genre.strip() for genre in x.split(',')]))

# Paso 2: Codificación one-hot de géneros
genres_encoded = pd.get_dummies(games['genres'].str.split(', ').apply(pd.Series).stack())

# Suma las columnas para obtener la codificación de género
genres_encoded = genres_encoded.groupby(level=0).sum()

# Paso 3: Función de recomendación
def recomendacion_juego(id_de_producto):
    try:
        # Encuentra el vector de género del juego de entrada
        juego_genero = genres_encoded.loc[df_games[df_games['id'] == id_de_producto].index[0]]

        # Calcula la similitud del coseno entre el juego de entrada y todos los juegos
        sim_scores = cosine_similarity([juego_genero], genres_encoded)[0]

        # Enumera las similitudes del coseno junto con los índices de los juegos
        sim_scores_with_indices = list(enumerate(sim_scores))

        # Ordena los juegos por similitud del coseno en orden descendente
        sim_scores_with_indices.sort(key=lambda x: x[1], reverse=True)

        # Obtiene los índices de los juegos más similares (excluyendo el propio juego)
        top_indices = [index for index, _ in sim_scores_with_indices[1:6]]

        # Obtiene los títulos de los juegos más similares
        top_games = games.iloc[top_indices]['title'].tolist()

        return top_games

    except Exception as e:
        return {"message": f"Error: {str(e)}"}


/var/folders/5w/6sy3cr4d4t35b_h01s9cd7mr0000gn/T/ipykernel_17188/2558606823.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games['genres'] = games['genres'].apply(lambda x: ', '.join([genre.strip() for genre in x.split(',')]))


In [89]:
id_producto = 761140  # Reemplaza esto con el ID del juego de entrada
juegos_recomendados = recomendacion_juego(id_producto)
print(juegos_recomendados)

['pixel puzzles 2: anime', 'world of cinema - directors cut', 'toy soldiers: complete', 'pixel puzzles 2: radical roach', 'aerial destruction']
